In [37]:
import pandas as pd
import os
import gc
import sys

dtype_dict = {
    'P_2': 'float16',
    'D_39': 'float16',
    'B_1': 'float16',
    'B_2': 'float16',
    'R_1': 'float16',
    'S_3': 'float16',
    'D_41': 'float16',
    'B_3': 'float16',
    'D_42': 'float16',
    'D_43': 'float16',
    'D_44': 'float16',
    'B_4': 'float16',
    'D_45': 'float16',
    'B_5': 'float16',
    'R_2': 'float16',
    'D_46': 'float16',
    'D_47': 'float16',
    'D_48': 'float16',
    'D_49': 'float16',
    'B_6': 'float16',
    'B_7': 'float16',
    'B_8': 'float16',
    'D_50': 'float16',
    'D_51': 'float16',
    'B_9': 'float16',
    'R_3': 'float16',
    'D_52': 'float16',
    'P_3': 'float16',
    'B_10': 'float16',
    'D_53': 'float16',
    'S_5': 'float16',
    'B_11': 'float16',
    'S_6': 'float16',
    'D_54': 'float16',
    'R_4': 'float16',
    'S_7': 'float16',
    'B_12': 'float16',
    'S_8': 'float16',
    'D_55': 'float16',
    'D_56': 'float16',
    'B_13': 'float16',
    'R_5': 'float16',
    'D_58': 'float16',
    'S_9': 'float16',
    'B_14': 'float16',
    'D_59': 'float16',
    'D_60': 'float16',
    'D_61': 'float16',
    'B_15': 'float16',
    'S_11': 'float16',
    'D_62': 'float16',
    'D_63': 'object',
    'D_64': 'object',
    'D_65': 'float16',
    'B_16': 'float16',
    'B_17': 'float16',
    'B_18': 'float16',
    'B_19': 'float16',
    'D_66': 'float16',
    'B_20': 'float16',
    'D_68': 'float16',
    'S_12': 'float16',
    'R_6': 'float16',
    'S_13': 'float16',
    'B_21': 'float16',
    'D_69': 'float16',
    'B_22': 'float16',
    'D_70': 'float16',
    'D_71': 'float16',
    'D_72': 'float16',
    'S_15': 'float16',
    'B_23': 'float16',
    'D_73': 'float16',
    'P_4': 'float16',
    'D_74': 'float16',
    'D_75': 'float16',
    'D_76': 'float16',
    'B_24': 'float16',
    'R_7': 'float16',
    'D_77': 'float16',
    'B_25': 'float16',
    'B_26': 'float16',
    'D_78': 'float16',
    'D_79': 'float16',
    'R_8': 'float16',
    'R_9': 'float16',
    'S_16': 'float16',
    'D_80': 'float16',
    'R_10': 'float16',
    'R_11': 'float16',
    'B_27': 'float16',
    'D_81': 'float16',
    'D_82': 'float16',
    'S_17': 'float16',
    'R_12': 'float16',
    'B_28': 'float16',
    'R_13': 'float16',
    'D_83': 'float16',
    'R_14': 'float16',
    'R_15': 'float16',
    'D_84': 'float16',
    'R_16': 'float16',
    'B_29': 'float16',
    'B_30': 'float16',
    'S_18': 'float16',
    'D_86': 'float16',
    'D_87': 'float16',
    'R_17': 'float16',
    'R_18': 'float16',
    'D_88': 'float16',
    'B_31': 'int64',
    'S_19': 'float16',
    'R_19': 'float16',
    'B_32': 'float16',
    'S_20': 'float16',
    'R_20': 'float16',
    'R_21': 'float16',
    'B_33': 'float16',
    'D_89': 'float16',
    'R_22': 'float16',
    'R_23': 'float16',
    'D_91': 'float16',
    'D_92': 'float16',
    'D_93': 'float16',
    'D_94': 'float16',
    'R_24': 'float16',
    'R_25': 'float16',
    'D_96': 'float16',
    'S_22': 'float16',
    'S_23': 'float16',
    'S_24': 'float16',
    'S_25': 'float16',
    'S_26': 'float16',
    'D_102': 'float16',
    'D_103': 'float16',
    'D_104': 'float16',
    'D_105': 'float16',
    'D_106': 'float16',
    'D_107': 'float16',
    'B_36': 'float16',
    'B_37': 'float16',
    'R_26': 'float16',
    'R_27': 'float16',
    'B_38': 'float16',
    'D_108': 'float16',
    'D_109': 'float16',
    'D_110': 'float16',
    'D_111': 'float16',
    'B_39': 'float16',
    'D_112': 'float16',
    'B_40': 'float16',
    'S_27': 'float16',
    'D_113': 'float16',
    'D_114': 'float16',
    'D_115': 'float16',
    'D_116': 'float16',
    'D_117': 'float16',
    'D_118': 'float16',
    'D_119': 'float16',
    'D_120': 'float16',
    'D_121': 'float16',
    'D_122': 'float16',
    'D_123': 'float16',
    'D_124': 'float16',
    'D_125': 'float16',
    'D_126': 'float16',
    'D_127': 'float16',
    'D_128': 'float16',
    'D_129': 'float16',
    'B_41': 'float16',
    'B_42': 'float16',
    'D_130': 'float16',
    'D_131': 'float16',
    'D_132': 'float16',
    'D_133': 'float16',
    'R_28': 'float16',
    'D_134': 'float16',
    'D_135': 'float16',
    'D_136': 'float16',
    'D_137': 'float16',
    'D_138': 'float16',
    'D_139': 'float16',
    'D_140': 'float16',
    'D_141': 'float16',
    'D_142': 'float16',
    'D_143': 'float16',
    'D_144': 'float16',
    'D_145': 'float16'}
ipython_vars = ["In", "Out", "exit", "quit", "get_ipython", "ipython_vars"]
categorical = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
def mem_usage_gb(df):
    return round((df.memory_usage(deep=True).sum()/1073741824), 2)
def file_mem_usage_gb(file):
    return round((os.stat(file).st_size/1073741824), 2)

In [38]:
train_data = pd.read_csv('train_data.csv', dtype=dtype_dict)
numerical = train_data.drop(columns= categorical).columns.to_list()

numerical.remove('customer_ID'); numerical.remove('S_2')

train_num_agg = train_data.groupby("customer_ID")[numerical].agg(['mean', 'std', 'min', 'max', 'last'])
train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
train_num_agg.reset_index(inplace = True)
train_num_agg.head(10)

customer_ID  P_2_mean   P_2_std   P_2_min   P_2_max  P_2_last  D_39_mean  D_39_std  D_39_min  D_39_max  D_39_last  B_1_mean   B_1_std   B_1_min   B_1_max  B_1_last  B_2_mean   B_2_std   B_2_min   B_2_max  B_2_last  R_1_mean   R_1_std   R_1_min   R_1_max  R_1_last  S_3_mean   S_3_std   S_3_min   S_3_max  S_3_last  D_41_mean  D_41_std  D_41_min  D_41_max  D_41_last  B_3_mean   B_3_std   B_3_min   B_3_max  B_3_last  D_42_mean  D_42_std  D_42_min  D_42_max  D_42_last  D_43_mean  D_43_std  D_43_min  D_43_max  D_43_last  D_44_mean  D_44_std  D_44_min  D_44_max  D_44_last  B_4_mean   B_4_std   B_4_min   B_4_max  B_4_last  D_45_mean  D_45_std  D_45_min  D_45_max  D_45_last  B_5_mean   B_5_std   B_5_min   B_5_max  B_5_last  R_2_mean   R_2_std   R_2_min   R_2_max  R_2_last  D_46_mean  D_46_std  D_46_min  D_46_max  D_46_last  D_47_mean  D_47_std  D_47_min  D_47_max  D_47_last  D_48_mean  D_48_std  D_48_min  D_48_max  D_48_last  D_49_mean  D_49_std  \
0  0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...  0.933594  0.024194  0.868652  0.960449  0.934570   0.010704  0.024440  0.001082  0.091492   0.009117  0.012009  0.006546  0.001930  0.021652  0.009384  1.004883  0.003248  1.000000  1.009766  1.007812  0.004509  0.003081  0.000263  0.009224  0.006104  0.113220  0.011665  0.098877  0.135010  0.135010   0.005020  0.003331  0.000685  0.009857   0.001604  0.006454  0.002942  0.000783  0.009865  0.007175        NaN       NaN       NaN       NaN        NaN        NaN       NaN       NaN       NaN        NaN   0.004673  0.002823  0.000630  0.009399   0.003258  0.040314  0.031726  0.000836  0.080994  0.070801   0.725586  0.009530  0.708984  0.740234   0.740234  0.146606  0.047200  0.060486  0.231689  0.231689  0.005753  0.002928  0.001436  0.009918  0.008308   0.377930  0.085663  0.230957  0.519531   0.420410   0.532715  0.006533  0.521484  0.541992   0.539551   0.240967  0.076889  0.135620  0.403564   0.192383        NaN       NaN   
1  00000fd6641609c6ece5454664794f0340ad84dddce9a2...  0.899902  0.022097  0.861328  0.929199  0.880371   0.215210  0.199123  0.002224  0.567383   0.178101  0.025650  0.027750  0.006710  0.109619  0.034698  0.991211  0.051519  0.819824  1.008789  1.003906  0.006245  0.002128  0.001023  0.008995  0.006912  0.120605  0.023842  0.089783  0.165527  0.165527   0.004993  0.002903  0.000728  0.009514   0.005550  0.005665  0.003355  0.000861  0.012863  0.005070        NaN       NaN       NaN       NaN        NaN   0.144531  0.169515  0.060638  0.525391   0.060638   0.004311  0.002976  0.000123  0.009872   0.008781  0.016754  0.011410  0.001098  0.046570  0.020630   0.256592  0.009294  0.239502  0.267334   0.266357  0.035461  0.043903  0.004074  0.165161  0.026993  0.006252  0.002645  0.000905  0.009560  0.004978   0.452148  0.013190  0.432373  0.471680   0.438721   0.392334  0.006657  0.382568  0.402832   0.402100   0.048218  0.031314  0.010117  0.106018   0.014694        NaN       NaN   
2  00001b22f846c82c51f6e3958ccd81970162bae8b007e8...  0.878418  0.028837  0.797852  0.904297  0.880859   0.004181  0.002759  0.000802  0.009705   0.009705  0.004387  0.002786  0.001472  0.009995  0.004284  0.815430  0.003485  0.811035  0.819824  0.812500  0.006622  0.001920  0.003540  0.009445  0.006451       NaN       NaN       NaN       NaN       NaN   0.006844  0.002683  0.001654  0.009560   0.003796  0.005493  0.002834  0.000627  0.009384  0.007195        NaN       NaN       NaN       NaN        NaN        NaN       NaN       NaN       NaN        NaN   0.013901  0.034419  0.000628  0.128174   0.000628  0.032349  0.022472  0.013138  0.092285  0.031052   0.236938  0.008914  0.222412  0.251709   0.251709  0.004620  0.003044  0.000215  0.008659  0.001557  0.006054  0.003007  0.001342  0.009583  0.001687   0.464355  0.060147  0.413086  0.646973   0.433594   0.328613  0.007143  0.318359  0.339600   0.339111   0.092285  0.060611  0.030228  0.255127   0.080383        NaN       NaN   
3  000041bdba6ecadd89a52d11886e8eaaec9325906c9723...  0.59912

In [39]:
train_cat_agg = train_data.groupby("customer_ID")[categorical].agg(['count', 'last', 'nunique'])
train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
train_cat_agg.reset_index(inplace = True)
train_cat_agg.drop(['customer_ID'], axis= 1, inplace= True)
agg_cat_cols = train_cat_agg.columns
train_cat_agg.head(10)

,B_30_count,B_30_last,B_30_nunique,B_38_count,B_38_last,B_38_nunique,D_114_count,D_114_last,D_114_nunique,D_116_count,D_116_last,D_116_nunique,D_117_count,D_117_last,D_117_nunique,D_120_count,D_120_last,D_120_nunique,D_126_count,D_126_last,D_126_nunique,D_63_count,D_63_last,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,13,0.0,1,13,2.0,1,13,1.0,1,13,0.0,1,13,4.0,1,13,0.0,1,13,1.0,1,13,CR,1,13,O,1,0,NaN,0,13,6.0,1
1,13,0.0,1,13,2.0,1,13,1.0,1,13,0.0,1,13,-1.0,1,13,0.0,2,13,1.0,1,13,CO,1,13,O,1,0,NaN,0,13,6.0,1
2,13,0.0,1,13,1.0,1,13,1.0,2,13,0.0,1,13,-1.0,1,13,0.0,1,13,1.0,1,13,CO,1,13,R,1,0,NaN,0,13,6.0,1
3,13,0.0,1,13,2.0,1,13,1.0,1,13,0.0,1,13,6.0,2,13,0.0,1,13,1.0,1,13,CO,1,13,O,1,0,NaN,0,13,3.0,3
4,13,0.0,1,13,1.0,2,13,1.0,1,13,0.0,1,13,4.0,1,13,0.0,1,13,1.0,1,13,CO,1,13,O,1,13,1.0,1,13,6.0,1
5,13,0.0,1,13,2.0,2,13,1.0,1,13,0.0,1,13,1.0,2,13,0.0,1,13,1.0,1,13,CO,1,13,R,1,0,NaN,0,13,6.0,1
6,13,0.0,3,13,3.0,3,11,0.0,1,11,0.0,1,11,2.0,1,11,0.0,1,11,0.0,1,13,CO,1,11,R,1,0,NaN,0,11,3.0,2
7,13,0.0,1,13,3.0,2,13,1.0,1,13,0.0,1,13,6.0,1,13,0.0,1,13,1.0,1,13,CO,1,13,R,1,0,NaN,0,13,5.0,1
8,13,1.0,1,13,7.0,3,13,0.0,1,13,0.0,1,13,-1.0,1,13,0.0,1,13,1.0,1,13,CO,1,13,U,1,13,1.0,1,13,6.0,2
9,13,0.0,1,13,1.0,1,13,0.0,2,13,0.0,1,13,4.0,1,13,0.0,1,13,1.0,1,13,CO,1,13,O,1,0,NaN,0,13,5.0,1


In [40]:
train_data = pd.concat([train_num_agg, train_cat_agg], axis= 1)
# train_data.to_csv('prep_catboost_train.csv', index=False)
print('Saved preprocessed train data')
print(f'Size of pandas DataFrame in GB: {mem_usage_gb(train_data)}')
print(f'Size of saved CSV in GB: {file_mem_usage_gb("prep_catboost_train.csv")}')
mem = {
    key: value
    for key, value in sorted(
        [
            (x, sys.getsizeof(globals().get(x)))
            for x in dir()
            if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
        ],
        key=lambda x: x[1],
        reverse=True,
    )
}
print(mem, round(sum(mem.values()) / 1e6, 2))

Saved preprocessed train data
Size of pandas DataFrame in GB: 1.41
Size of saved CSV in GB: 3.47
{'train_data': 1508728442, 'train_num_agg': 1366184145, 'train_cat_agg': 142544441, 'dtype_dict': 9312, 'agg_cat_cols': 3325, 'numerical': 1488, 'DataFrame': 1064, 'LabelEncoder': 1064, 'PCA': 1064, 'mem': 640, 'categorical': 152, 'accuracy_score': 136, 'file_mem_usage_gb': 136, 'mem_usage_gb': 136, 'total_mem_usage_mb': 136, 'train_test_split': 136, 'np': 72, 'pd': 72, 'column': 61, 'X_num_agg': 48} 3017.48


In [41]:
del train_data
del train_cat_agg
del train_num_agg
gc.collect()
print('Delete train dataset and do gc')
mem = {
    key: value
    for key, value in sorted(
        [
            (x, sys.getsizeof(globals().get(x)))
            for x in dir()
            if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
        ],
        key=lambda x: x[1],
        reverse=True,
    )
}
print(mem, round(sum(mem.values()) / 1e6, 2))

Delete train dataset and do gc
{'dtype_dict': 9312, 'agg_cat_cols': 3325, 'numerical': 1488, 'DataFrame': 1064, 'LabelEncoder': 1064, 'PCA': 1064, 'mem': 640, 'categorical': 152, 'accuracy_score': 136, 'file_mem_usage_gb': 136, 'mem_usage_gb': 136, 'total_mem_usage_mb': 136, 'train_test_split': 136, 'np': 72, 'pd': 72, 'column': 61, 'X_num_agg': 48} 0.02


In [42]:
test_data = pd.read_csv('test_data.csv', dtype=dtype_dict)
test_agg_num = test_data.groupby("customer_ID")[numerical].agg(['mean', 'std', 'min', 'max', 'last'])
test_agg_cat = test_data.groupby("customer_ID")[categorical].agg(['count', 'last', 'nunique'])
test_agg_num.columns = ['_'.join(x) for x in test_agg_num.columns]
test_agg_num.reset_index(inplace = True)
test_agg_cat.columns = ['_'.join(x) for x in test_agg_cat.columns]
test_agg_cat.reset_index(inplace = True)
test_agg_cat.drop(['customer_ID'], axis= 1, inplace= True)

In [43]:
test_data = pd.concat([test_agg_num, test_agg_cat], axis= 1)
# test_data.to_csv('prep_catboost_test.csv', index=False)
print('Saved preprocessed test data')
print(f'Size of pandas DataFrame in GB: {mem_usage_gb(test_data)}')
print(f'Size of saved CSV in GB: {file_mem_usage_gb("prep_catboost_test.csv")}')
mem = {
    key: value
    for key, value in sorted(
        [
            (x, sys.getsizeof(globals().get(x)))
            for x in dir()
            if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
        ],
        key=lambda x: x[1],
        reverse=True,
    )
}
print(mem, round(sum(mem.values()) / 1e6, 2))

Saved preprocessed test data
Size of pandas DataFrame in GB: 2.83
Size of saved CSV in GB: 7.04
{'test_data': 3039431934, 'test_agg_num': 2752596861, 'test_agg_cat': 286835217, 'dtype_dict': 9312, 'agg_cat_cols': 3325, 'numerical': 1488, 'DataFrame': 1064, 'LabelEncoder': 1064, 'PCA': 1064, 'mem': 640, 'categorical': 152, 'accuracy_score': 136, 'file_mem_usage_gb': 136, 'mem_usage_gb': 136, 'total_mem_usage_mb': 136, 'train_test_split': 136, 'np': 72, 'pd': 72, 'column': 61, 'X_num_agg': 48} 6078.88


In [44]:
del test_data
del test_agg_cat
del test_agg_num
gc.collect()
print('Delete train dataset and do gc')
mem = {
    key: value
    for key, value in sorted(
        [
            (x, sys.getsizeof(globals().get(x)))
            for x in dir()
            if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
        ],
        key=lambda x: x[1],
        reverse=True,
    )
}
print(mem, round(sum(mem.values()) / 1e6, 2))

Delete train dataset and do gc
{'dtype_dict': 9312, 'agg_cat_cols': 3325, 'numerical': 1488, 'DataFrame': 1064, 'LabelEncoder': 1064, 'PCA': 1064, 'mem': 640, 'categorical': 152, 'accuracy_score': 136, 'file_mem_usage_gb': 136, 'mem_usage_gb': 136, 'total_mem_usage_mb': 136, 'train_test_split': 136, 'np': 72, 'pd': 72, 'column': 61, 'X_num_agg': 48} 0.02
